<a href="https://colab.research.google.com/github/theMADAIguy/STOCK-PREDICTION-USING-SVM-REGRESSION/blob/master/text_tutorial_pytorch_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchtext
from torchtext import data
from torchtext import datasets
import random
import torch.nn as nn
import torch.optim as optim

#Constants
SEED = 1234
BATCH_SIZE=64
EPOCHS=10


torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/
PATH = './text_rnn_model.pt'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [2]:
TEXT=data.Field(tokenize='spacy', include_lengths=True)
LABEL=data.LabelField(dtype=torch.float)

trainSet, testSet = datasets.IMDB.splits(TEXT, LABEL)
testSet, validSet = testSet.split(random_state=random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 197k/84.1M [00:00<00:50, 1.66MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 63.1MB/s]


In [3]:
TEXT.build_vocab(trainSet, max_size= MAX_SIZE_VOCAB, vectors='glove.6B.100d', unk_init=torch.Tensor.normal_)
LABEL.build_vocab(trainSet)

.vector_cache/glove.6B.zip: 862MB [06:27, 2.23MB/s]                           
100%|█████████▉| 399533/400000 [00:15<00:00, 26665.18it/s]

In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((trainSet, validSet, testSet), batch_size=BATCH_SIZE, device=device, sort_within_batch=True)

In [0]:
class RNN(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, vocab_size, dropout, bidirectional, n_layers, pad_idx):
    super().__init__()

    self.embedding=nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
    self.rnn=nn.LSTM(embedding_dim, hidden_dim,num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
    self.fc=nn.Linear(hidden_dim * 2, output_dim)
    self.dropout=nn.Dropout(dropout)

  def forward(self, text, text_lenghts):
    embedded=self.dropout(self.embedding(text))
    packed_embedding=nn.utils.rnn.pack_padded_sequence(embedded, text_lenghts)
    output, (hidden, cell)=self.rnn(packed_embedding)
    hidden=self.dropout(torch.cat((hidden[-1,:,:], hidden[-2,:,:]), dim=1))
    return self.fc(hidden)

In [48]:
INPUT_DIMENSION=len(TEXT.vocab)
EMBEDDING_DIMENSION=100
HIDDEN_DIMENSION=256
OUTPUT_DIMENSION=1
DROPOUT=0.5
BIDIRECTIONAL=True
N_LAYERS=2
PAD_IDX=TEXT.vocab.stoi[TEXT.pad_token]
MAX_SIZE_VOCAB=25002

model=RNN(INPUT_DIMENSION,EMBEDDING_DIMENSION, HIDDEN_DIMENSION, OUTPUT_DIMENSION, MAX_SIZE_VOCAB, DROPOUT, BIDIRECTIONAL, N_LAYERS, PAD_IDX)
print(model)

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [49]:
pre_trained_embedding=TEXT.vocab.vectors
print(pre_trained_embedding.shape)
model.embedding.weight.data.copy_(pre_trained_embedding)
UNK_IDX=TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX]=torch.zeros(EMBEDDING_DIMENSION)
model.embedding.weight.data[PAD_IDX]=torch.zeros(EMBEDDING_DIMENSION)

torch.Size([25002, 100])


In [0]:
optimizer=optim.Adam(model.parameters())
criterion=nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def accuracy(predicted, labels):

  rounded=torch.round(torch.sigmoid(predicted))
  correct=(rounded==labels).float()
  acc=correct.sum()/len(rounded)
  return acc

In [0]:
def train(model, iterator, optimizer, criterion):

  epoch_loss=0.0
  epoch_acc=0.0

  model.train()

  for batch in iterator:
    optimizer.zero_grad()
    text, text_lengths = batch.text
    output=model(text, text_lengths).squeeze(1)
    loss=criterion(output, batch.label)
    acc=accuracy(output, batch.label)
    loss.backward()
    optimizer.step()

    epoch_loss+=loss.item()
    epoch_acc+=acc.item()

  return epoch_loss/len(iterator), epoch_acc / len(iterator)


In [0]:
def evaluate(model, iterator, optimizer, criterion):

  epoch_loss=0.0
  epoch_acc=0.0

  model.eval()

  with torch.no_grad():

    for batch in iterator:
      text, text_lengths = batch.text
      output=model(text, text_lengths).squeeze(1)
      loss=criterion(output, batch.label)
      acc=accuracy(output, batch.label)
      
      epoch_loss+=loss.item()
      epoch_acc+=acc.item()

  return epoch_loss/len(iterator), epoch_acc / len(iterator)




In [55]:
best_valid_loss=float('inf')
for epoch in range(EPOCHS):

  train_loss, train_acc=train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc=evaluate(model, valid_iterator, optimizer, criterion)

  if(valid_loss < best_valid_loss):
    best_valid_loss=valid_loss
    torch.save(model.state_dict(), PATH)
  
  print("Epoch: ", epoch, "train_loss: ", train_loss, "valid_loss: ", valid_loss, "train_acc: ", train_acc, "valid_acc: ", valid_acc)

Epoch:  0 train_loss:  0.6718334235498667 valid_loss:  0.6571473418656042 train_acc:  0.5867167519181585 valid_acc:  0.6202330508474576
Epoch:  1 train_loss:  0.6199127686450548 valid_loss:  0.56934842214746 train_acc:  0.6587675831202046 valid_acc:  0.7292108050847458
Epoch:  2 train_loss:  0.6158705341541554 valid_loss:  0.4776454699241509 train_acc:  0.6553388747107952 valid_acc:  0.780941031241821
Epoch:  3 train_loss:  0.37520733925387684 valid_loss:  0.3764059850472515 train_acc:  0.8392343350383632 valid_acc:  0.8620674438395742
Epoch:  4 train_loss:  0.2636686168096559 valid_loss:  0.27481875682281237 train_acc:  0.8924952045730923 valid_acc:  0.8832980227672448
Epoch:  5 train_loss:  0.21901952532474953 valid_loss:  0.26300423674411694 train_acc:  0.9156010230179028 valid_acc:  0.8942884888689396
Epoch:  6 train_loss:  0.19316890298405573 valid_loss:  0.27388044333053846 train_acc:  0.9254635549567239 valid_acc:  0.8964071329367362
Epoch:  7 train_loss:  0.16989504102893802 va

In [56]:
model.load_state_dict(torch.load(PATH))

test_loss, test_acc=evaluate(model, test_iterator, optimizer, criterion)
print("test_loss: ", test_loss, "test_acc: ", test_acc*100)

test_loss:  0.2542279395852646 test_acc:  89.66208291749884
